# CPSC 330 Lecture 10

#### Lecture plan

- 👋
- **Turn on recording**
- Announcements (5 min)
- Dataset of the week: regression! (20 min)
- Linear regression intro (5 min)
- Regression score functions: mean squared error and R^2 (10 min)
- Break (5 min)
- Exploring `alpha` (5 min)
- Transforming the targets (20 min)
- Ensembling with regression (5 min)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, cross_val_score, cross_validate
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.dummy import DummyRegressor
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import Ridge

In [ ]:
from pandas_profiling import ProfileReport

In [ ]:
plt.rcParams['font.size'] = 16

## Announcements

Homework assignments:

- hw4 deadline passed
- hw5 coming today, due Monday at 11:59pm
- After that a 2-week break from hw 

Midterm:

- a

# TODO

- make some feature; why not price per square foot? (because it uses the target!)
- Also: MoSold (month sold) feature - to discuss?
- https://scikit-learn.org/stable/modules/generated/sklearn.compose.TransformedTargetRegressor.html


## Data set of the week (20 min)

This week in lecture we will be focussing on the [Kaggle House Prices dataset](https://www.kaggle.com/c/home-data-for-ml-course/). As usual, to run this notebook you'll need to download the data. Unzip the data. Rename `train.csv` to `housing.csv` and move it into the data directory of this repo. (For this dataset, train and test have already been separated. The "test" data they provide is actually what we call deployment. They only provide the X, not the y. So we couldn't actually use Kaggle's test set as our test set, since we need the y values for that.)

In [ ]:
df = pd.read_csv("data/housing.csv")

In [ ]:
df_train, df_test = train_test_split(df, test_size=0.1, random_state=123)

In [ ]:
df_train.head()

- Here, the target is `SalePrice`. Note that this is numeric, not categorical.
- In this case, we call the task **regression** (as opposed to classification).

In [ ]:
df_train.shape

In [ ]:
# df_train.info()

#### Step 1: `pandas_profiler`

In [ ]:
profile = ProfileReport(df_train, title='Pandas Profiling Report') #, minimal=True)

In [ ]:
profile.to_notebook_iframe()

- From here we can see a mix of feature types, and a bunch of missing values. 
- Now, I do some of the dirty work...
- There are a bunch of ordinal features using the same scale: excellent, good, average, etc.
  - These I'm calling `ordinal_features_reg`.
- There are a bunch more ordinal features using different scales.
  - These I'm calling `ordinal_features_oth`
  - To save time I'm ignoring the ordinality and just encoding them as categorical.

In [ ]:
target               = ['SalePrice']
drop_features        = ['Id']
numeric_features     = ['LotFrontage', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 
                        'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 
                        'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 
                        'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 
                        'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 'GarageCars', 
                        'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 
                        'ScreenPorch', 'PoolArea', 'MiscVal', 'YrSold']
ordinal_features_reg = ['ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond', 'HeatingQC', 
                        'KitchenQual', 'FireplaceQu', 'GarageQual', 'GarageCond', 'PoolQC']
ordinal_features_oth = ['BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 
                        'Functional',  'Fence']
categorical_features = list(set(df_train.columns) - set(target) - set(drop_features) - 
                            set(numeric_features) - 
                            set(ordinal_features_reg) - set(ordinal_features_oth))
all_features = numeric_features + ordinal_features_reg + categorical_features + ordinal_features_oth

ordering = ['Po', 'Fa', 'TA', 'Gd', 'Ex'] # if N/A it will just impute something, per below

#### Step 2: data quality

- We'll just use `SimpleImputer` again.
- In reality we'd want to go through this more carefully.
- We may also want to drop some columns that are almost entirely missing.
- We could also check for outliers, and do other EDA.

In [ ]:
imputers = [
    ('numeric', SimpleImputer(strategy='median'), numeric_features),
    ('ordinal', SimpleImputer(strategy='most_frequent'), ordinal_features_reg),
    ('categor', SimpleImputer(strategy='constant', fill_value="?"), 
     categorical_features + ordinal_features_oth)]
# NOTE: the order here must match the order of all_features

In [ ]:
impute_transformer = ColumnTransformer(transformers=imputers)

In [ ]:
impute_transformer.fit(df_train);

In [ ]:
df_train_imp = pd.DataFrame(impute_transformer.transform(df_train), index=df_train.index, columns=all_features)
df_valid_imp = pd.DataFrame(impute_transformer.transform(df_valid), index=df_valid.index, columns=all_features)
df_test_imp  = pd.DataFrame(impute_transformer.transform(df_test),  index=df_test.index,  columns=all_features)

In [ ]:
df_train_imp.head()

#### Step 3: feature transformations

In [ ]:
feature_transformers = [
    ('scale',  StandardScaler(), numeric_features),
    ('ord',    OrdinalEncoder(categories=[ordering for i in ordinal_features_reg]), ordinal_features_reg),
    ('ohe',    OneHotEncoder(drop='first', sparse=False), categorical_features + ordinal_features_oth) ]

In [ ]:
feature_preprocessor = ColumnTransformer(transformers=feature_transformers)

In [ ]:
feature_preprocessor.fit(df_train_imp);

In [ ]:
new_columns = numeric_features + ordinal_features_reg + list(feature_preprocessor.named_transformers_['ohe'].get_feature_names(categorical_features + ordinal_features_oth))
new_columns;
# NOTE: the order here must match the order above

In [ ]:
df_train_imp_encode = pd.DataFrame(feature_preprocessor.transform(df_train_imp), index=df_train_imp.index, columns=new_columns)
df_valid_imp_encode = pd.DataFrame(feature_preprocessor.transform(df_valid_imp), index=df_valid_imp.index, columns=new_columns)
df_test_imp_encode  = pd.DataFrame(feature_preprocessor.transform(df_test_imp),  index=df_test_imp.index,  columns=new_columns)

What happened here?

- We fit the `OneHotEncoder` on the training data.
- Due to the splits, none of the training examples had these categories, but the validation/test set has them.
- How do we want to handle this? 
- It depends!
- Option 1: fit the OHE on _all_ data.
  - But isn't this violating the Golden Rule??
  - Well, do we know the categories in advance?
  - Or is there a chance we'll see a new category in deployment? 
- Option 2: have a special category for "unknown".
  - We can reserve all zeros, but then we can't drop the first.
  - Let's do Option 2, to be safe.

In [ ]:
feature_transformers = [
    ('scale',  StandardScaler(), numeric_features),
    ('ord',    OrdinalEncoder(categories=[ordering for i in ordinal_features_reg]), ordinal_features_reg),
    # note changes           VVVVVVVVVV       and     VVVVVVVVVVVVVVVVVVVVVVV
    ('ohe',    OneHotEncoder(drop=None, sparse=False, handle_unknown='ignore'), categorical_features + ordinal_features_oth)]

In [ ]:
feature_preprocessor = ColumnTransformer(transformers=feature_transformers)

In [ ]:
feature_preprocessor.fit(df_train_imp);

In [ ]:
new_columns = numeric_features + ordinal_features_reg + list(feature_preprocessor.named_transformers_['ohe'].get_feature_names(categorical_features + ordinal_features_oth))
new_columns;

In [ ]:
X_train_imp_encode = feature_preprocessor.transform(df_train_imp)
X_valid_imp_encode = feature_preprocessor.transform(df_valid_imp)
X_test_imp_encode  = feature_preprocessor.transform(df_test_imp)

df_train_imp_encode = pd.DataFrame(X_train_imp_encode, index=df_train_imp.index, columns=new_columns)
df_valid_imp_encode = pd.DataFrame(X_valid_imp_encode, index=df_valid_imp.index, columns=new_columns)
df_test_imp_encode  = pd.DataFrame(X_test_imp_encode,  index=df_test_imp.index,  columns=new_columns)

In [ ]:
y_train = df_train["SalePrice"]
y_valid = df_valid["SalePrice"]
y_test  = df_test["SalePrice"]

In [ ]:
# new_columns

In [ ]:
X_train_imp_encode.shape

#### Step 4: `DummyRegressor`

In [ ]:
dummy = DummyRegressor()
dummy.fit(X_train_imp_encode, y_train);

In [ ]:
dummy.score(X_train_imp_encode, y_train)

In [ ]:
dummy.score(X_valid_imp_encode, y_valid)

Wait, a negative score??

## Linear regression intro (5 min)

- Linear regression is one of the most basic and popular ML/statistical techniques. 
- We already saw logistic regression for classification - these are close cousins.
- Both are very _interpretable_.
- For more depth, see CPSC 340, STAT 306, and various other STAT courses.
- In scikit-learn, we can do:

In [ ]:
LinearRegression();

However, I am going to recommend always using:

In [ ]:
Ridge();

- `Ridge` is more flexible than `LinearRegression` because it has a hyperparameter `alpha`:

In [ ]:
lr_vanilla = LinearRegression()
lr_vanilla.fit(X_train_imp_encode, y_train);

In [ ]:
lr_vanilla_preds = lr_vanilla.predict(X_valid_imp_encode)
lr_vanilla_preds[:10]

In [ ]:
lr_vanilla_preds.max()

In [ ]:
lr_vanilla_preds.min()

- One prediction is for $\$10^{15}$ !!
- One prediction is for $ - \$10^{15}$ !!!!
- This happened because we have "collinear features" and there are "numerical issues" (see also CPSC 302).
- `Ridge` "solves" this issue:

In [ ]:
lr = Ridge()
lr.fit(X_train_imp_encode, y_train);

In [ ]:
lr.predict(X_valid_imp_encode)[:10]

- `Ridge` has a hyperparameter `alpha`.
- This is like `C` in `LogisticRegression` but, annoyingly, `alpha` is the _inverse_ of `C`.
  - That is, large `C` is like small `alpha` and vice versa.
- We can (approximately) recover the original `LinearRegression` by setting `alpha=0` (but I don't recommend this).
- TL;DR don't use `LinearRegression`
  - Unless you know what you're doing, but then use [statsmodels](https://www.statsmodels.org/stable/index.html) or R instead of sklearn. 

Let's compare with the random forest:

In [ ]:
lr = Ridge()
lr.fit(X_train_imp_encode, y_train_log);

In [ ]:
print("Training MAPE: %.1f%%" % mape(y_train, np.exp(lr.predict(X_train_imp_encode))))

In [ ]:
print("Validation MAPE: %.1f%%" % mape(y_valid, np.exp(lr.predict(X_valid_imp_encode))))

In [ ]:
rf = RandomForestRegressor(random_state=111)
rf.fit(X_train_imp_encode, np.log(y_train));

In [ ]:
print("Training MAPE: %.1f%%" % mape(y_train, np.exp(rf.predict(X_train_imp_encode))))

In [ ]:
print("Validation MAPE: %.1f%%" % mape(y_valid, np.exp(rf.predict(X_valid_imp_encode))))

- So far, the linear model seems to be doing better. 
- Note that it's also a _simpler_ model.
- Like logistic regression, the intuition is that "the complexity grows as you add more features". 

## Regression score functions: mean squared error and R^2 (10 min)

We aren't doing classification anymore, so we can't just check for equality:

In [ ]:
dummy.predict(X_train_imp_encode) == y_train

In [ ]:
dummy.predict(X_train_imp_encode)

In [ ]:
y_train.values

- We need a score that reflect how right/wrong each prediction is.

#### Mean squared error (MSE)

A common measure is mean squared error:

In [ ]:
preds = dummy.predict(X_train_imp_encode)

In [ ]:
np.mean((y_train - preds)**2)

This is also implemented in sklearn:

In [ ]:
mean_squared_error(y_train, preds)

- Perfect predictions would have MSE=0. 

In [ ]:
mean_squared_error(y_train, y_train)

- But is the above score good or bad? 
  - It depends on the scale of the targets.
  - If we were working in cents instead of dollars, our MSE would be 10,000x ($100^2$) higher!

In [ ]:
np.mean((y_train*100 - preds*100)**2)

- A common score is the $R^2$. 
- You can [read about it](https://en.wikipedia.org/wiki/Coefficient_of_determination) if interested.
- Intuition: mean squared error, but flipped (higher is better), and normalized so the max is 1.
- Key points:
  - The maximum is 1 for perfect predictions
  - Negative values are very bad: "worse than `DummyRegressor`" (very bad)

In [ ]:
r2_score(y_train, preds)

In [ ]:
r2_score(y_train, y_train)

In [ ]:
r2_score(y_valid, dummy.predict(X_valid_imp_encode))

This is the score that sklearn uses by default when you call `score()`:

In [ ]:
dummy.score(X_train_imp_encode, y_train)

In [ ]:
dummy.score(X_valid_imp_encode, y_valid)

(optional) Warning: MSE is "reversible" but $R^2$ is not:

In [ ]:
mean_squared_error(y_train, preds)

In [ ]:
mean_squared_error(preds, y_train)

In [ ]:
r2_score(y_train, preds)

In [ ]:
r2_score(preds, y_train)

- When you call `fit` it minimizes MSE / maximizes $R^2$ by default.
- Just like in classification, this isn't always what you want!!
- More on this later.

## Break (5 min)

<br><br>

## Exploring `alpha` (5 min)

- The `alpha` hyperparameter controls the fundamental tradeoff as usual.
  - Smaller `alpha`: lower training error.
  - Larger `alpha`: lower approximation error (hopefully). 
- General intuition: larger `alpha` leads to **smaller coefficients**. 
  - Smaller coefficients mean the predictions are less sensitive to changes in the data.
  - Hence less chance of overfitting (seeing big dependencies when you shouldn't).
- Let's test this out:

In [ ]:
lr = Ridge(alpha=0.01)
lr.fit(X_train_imp_encode, y_train_log);
np.max(np.abs(lr.coef_))

In [ ]:
lr = Ridge(alpha=1)
lr.fit(X_train_imp_encode, y_train_log);
np.max(np.abs(lr.coef_))

In [ ]:
lr = Ridge(alpha=1000)
lr.fit(X_train_imp_encode, y_train_log);
np.max(np.abs(lr.coef_))

- Note that it will not make all the coefficients smaller in the same proportion!
- The order of the coefficients might change. 

In [ ]:
lr = Ridge(alpha=0.01)
lr.fit(X_train_imp_encode, y_train_log);
df_train_imp_encode.columns[np.argmax(np.abs(lr.coef_))]

In [ ]:
lr = Ridge(alpha=1)
lr.fit(X_train_imp_encode, y_train_log);
df_train_imp_encode.columns[np.argmax(np.abs(lr.coef_))]

In [ ]:
lr = Ridge(alpha=100)
lr.fit(X_train_imp_encode, y_train_log);
df_train_imp_encode.columns[np.argmax(np.abs(lr.coef_))]

Let's try tuning `alpha`:

In [ ]:
alphas = 10.0**np.arange(-1.5,4.5,0.5)
train_errs = []
valid_errs = []
for alpha in alphas:

    lr = Ridge(alpha=alpha)
    lr.fit(X_train_imp_encode, y_train_log);
    train_errs.append(mape(y_train, np.exp(lr.predict(X_train_imp_encode))))
    valid_errs.append(mape(y_valid, np.exp(lr.predict(X_valid_imp_encode))))

In [ ]:
plt.semilogx(alphas, train_errs, label="train");
plt.semilogx(alphas, valid_errs, label="valid");
plt.legend();
plt.xlabel('alpha');
plt.ylabel('MAPE');

In [ ]:
best_alpha = alphas[np.argmin(valid_errs)]
best_alpha

In [ ]:
train_errs = []
valid_errs = []
for alpha in alphas:

    lr = Ridge(alpha=alpha)
    lr.fit(X_train_imp_encode, y_train);
    train_errs.append(np.sqrt(mean_squared_error(y_train, lr.predict(X_train_imp_encode))))
    valid_errs.append(np.sqrt(mean_squared_error(y_valid, lr.predict(X_valid_imp_encode))))

In [ ]:
plt.semilogx(alphas, train_errs, label="train");
plt.semilogx(alphas, valid_errs, label="valid");
plt.legend();
plt.xlabel('alpha');
plt.ylabel('MSE');

In [ ]:
best_alpha = alphas[np.argmin(valid_errs)]
best_alpha

- These are interesting curves, because the validation error is less than the training error.
- In short, that is because when `alpha` is large we are no longer directly minimizing training error.
- It seems `alpha=100` is the best choice here. 

In [ ]:
lr = Ridge(alpha=100)
lr.fit(X_train_imp_encode, y_train_log);

In [ ]:
print("Training MAPE: %.1f%%" % mape(y_train, np.exp(lr.predict(X_train_imp_encode))))

In [ ]:
print("Validation MAPE: %.1f%%" % mape(y_valid, np.exp(lr.predict(X_valid_imp_encode))))

- I'm sure one could do a lot better on this dataset, but 9% MAPE is a start!

## Transforming the targets (15 min)

Let's try something more serious than `DummyRegressor`:

In [ ]:
rf = RandomForestRegressor(random_state=111)
rf.fit(X_train_imp_encode, y_train);

In [ ]:
rf.score(X_train_imp_encode, y_train)

In [ ]:
rf.score(X_valid_imp_encode, y_valid)

In [ ]:
mse_valid = mean_squared_error(y_valid, rf.predict(X_valid_imp_encode))
mse_valid

- In this case, we are actually interested in the original units of dollars.
- MSE is in units of "squared dollars"
- We can take the square root of the this to get back to dollars:

In [ ]:
rmse_valid = np.sqrt(mse_valid)
rmse_valid

So, on average we're off by about \$25000. Is this good?

<br><br><br><br><br><br>

- For a house worth \\$500k, it seems reasonable! That's 5% error.
- For a house worth \\$50k, that is terrible. It's 50% error.

In [ ]:
plt.hist(y_train, bins=100);

- Indeed, we have both of these cases in our dataset.
- Can we compute percent error?

In [ ]:
pred_train = rf.predict(X_train_imp_encode)
pred_valid = rf.predict(X_valid_imp_encode)

In [ ]:
percent_errors = (pred_train - y_train)/y_train
percent_errors

- These are both positive (predict too high) and negative (predict too low).
- We can look at the absolute percent error:

In [ ]:
np.abs(percent_errors)

And, like MSE, we can take the average over examples. This is called **mean absolute percent error (MAPE)**.

In [ ]:
def mape(true, pred):
    return 100.*np.mean(np.abs((pred - true)/true))

In [ ]:
mape(y_train, pred_train)

In [ ]:
mape(y_valid, pred_valid)

- Ok, this is quite interpretable.
- On average, we have 11% error. Good to know. 

- ... but wait a minute, why are we minimizing MSE if we care about MAPE??
- When minimizing MSE, **the expensive houses will dominate** because they have the biggest error.
- The model would rather do \\$50 k worse on a cheap place and \\$60k better on an expensive place.
  - But this would make the MAPE much worse!
- Key idea: **log transform the targets**.
  - That is, transform $y\rightarrow \log(y)$.
- Why?
- Let's assume we have two values, the prediction and the true value.
- We log transform them and look at the squared error:

$$\begin{align}(\log y_\text{pred} - \log y_\text{true})^2 =\left(\log \frac{y_\text{pred}}{y_\text{true}}\right)^2  \end{align}$$

But what is the absolute percent error? It is

$$\left| \frac{y_\text{pred} - y_\text{true}}{y_\text{true}} \right| = \left| \frac{y_\text{pred}}{y_\text{true}} -1 \right|$$

This is a bit hand-wavy (and maybe I'll nail this down later), but for now: both are minimized (equal to zero) when $\frac{y_\text{pred}}{y_\text{true}}=1$, so both are trying to make $\frac{y_\text{pred}}{y_\text{true}}$ close to $1$.

In [ ]:
plt.hist(y_train, bins=100);

In [ ]:
plt.hist(np.log(y_train), bins=100);

In [ ]:
rf_log = RandomForestRegressor(random_state=111)
rf_log.fit(X_train_imp_encode, np.log(y_train));

In [ ]:
rf_log.predict(X_train_imp_encode)

These are log predictions. We can `exp` them to get back to dollars:

In [ ]:
pred_train_log_exp = np.exp(rf_log.predict(X_train_imp_encode))
pred_valid_log_exp = np.exp(rf_log.predict(X_valid_imp_encode))

In [ ]:
pred_train_log_exp

How is the MSE on the training set?

In [ ]:
mean_squared_error(pred_train, y_train) # ORIGINAL model

In [ ]:
mean_squared_error(pred_train_log_exp, y_train) # log transformed training

- The MSE got worse
  - That makes sense because we're no longer optimizing for MSE.
- Let's look at the MAPE.

In [ ]:
mape(y_train, pred_train)

In [ ]:
mape(y_train, pred_train_log_exp)

- The MAPE got better!
- We can do the same for the validation set:

In [ ]:
mape(y_valid, pred_valid)

In [ ]:
mape(y_valid, pred_valid_log_exp)

- Here we get a small benefit, but it could be very large in some cases.
- Also, the model's interpretation can often be better in the log transformed case.
- For linear regression: 
  - 1 more bedroom increases price by \\$50K vs
  - 1 more bedroom increases price by 5%.


- Note that this assumes the $y$-values are positive, which is true in this case.
  - There is still a problem if one of the $y$-values is zero, so it's common to do $\log(1+y)$ instead of $\log(y)$.
  - There is even a numpy function to do this for you: [`log1p`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.log1p.html) (better handing of floating point issues for small numbers -- not too relevant to us here).

## Ensembling with Regression (5 min)
mention how ensembling works with regression

1. show that it's the actual average
2. show how the LR coefs are now Ridge coefs.
3. Stacking doesn't have to use predict_proba - well, there is no predict_proba!

## Linear regression True/False (Piazza)

1. If the first coefficient is 5, that means increasing your first feature by 1 increases the prediction by 5.
2. Since the `PoolArea` has a positive coefficient, expanding my pool will get me a higher price when I sell my house.
3. Larger values of `alpha` are probably more useful when I have lots of features.
4. log-transforming the targets (and re-fitting) is equivalent to log-transforming the coefficients. 
5. In regression, one should use MAPE instead of MSE when relative (percent) error matters more than absolute error.